In [6]:
import torch
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

In [14]:
pr_qst = pd.DataFrame([ ## dataset
    {"question": "What is the historical significance of the mosaic in the apse of the Church of Panagia Aggeloktisti and in what century was it created?",
     "model_response": "",
     },
    {"question": "What architectural features distinguish the Church of Panagia Aggeloktisti and what periods of construction does it unite?",
     "model_response": "",
     },
    {"question": "What does the name Aggeloktisti mean and where does it come from?",
     "model_response": "",
     },
    {"question": "What chapels were added to the church in the Middle Ages and what was their purpose?",
     "model_response": "",
     },
    {"question": "How does the Church of Panagia Aggeloktisti demonstrate the cultural and artistic links between different regions of the Mediterranean?",
     "model_response": "",
     },
    {"question": "What arguments are put forward for the inclusion of the Church of Panagia Aggeloktisti in the UNESCO World Heritage List under criterion (i)?",
     "model_response": "",
     },
    {"question": "What conservation measures have been taken since 1952 to preserve the mosaics and other elements of the church?",
     "model_response": "",
     },
    {"question": "How does the church reflect the religious and spiritual life of the local community over the centuries?",
     "model_response": "",
    },
    {"question": "How does the church of Panagia Aggeloktisti maintain its authenticity and integrity despite the passing centuries and the restorations carried out?",
     "model_response": "",
    }
])

In [15]:
llama = "openlm-research/open_llama_3b"
llama_model = AutoModelForCausalLM.from_pretrained(
    llama,
    torch_dtype=torch.float16, ## gpu support
    device_map="auto",
    )
llama_tokenizer = AutoTokenizer.from_pretrained(llama)


In [23]:
def gen(question, model, tokenizer):
    prompt = f"Question: {question}\nAnswer:"
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **inputs,
        max_length=250,
        num_return_sequences=1,
        do_sample=False, ## variety, turn off for now
        top_p=0.95,
        temperature=0.7, ## temp
        pad_token_id=tokenizer.eos_token_id,
    )
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    answer = answer.split("Answer:")[-1].strip()
    return answer


llama_responses = []
for question in pr_qst["question"]:
    response = gen(question, llama_model, llama_tokenizer)
    llama_responses.append(response)

llama_dataframe = pr_qst.copy()
llama_dataframe["model_response"] = llama_responses

In [24]:
llama_dataframe["model_response"]

,model_response
0,The mosaic in the apse of the Church of Panagi...
1,The Church of Panagia Aggeloktisti is a three-...
2,Aggeloktisti is a name that means “angel-keepe...
3,A church is a building that is used for the ce...
4,The Church of Panagia Aggeloktisti is a unique...
5,The Church of Panagia Aggeloktisti is a unique...
6,The mosa
7,The church is the most important building in t...
8,The church of Panagia Aggeloktisti is a monume...
